In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit,concat,concat_ws,when,to_date,col,udf
from pyspark.sql.types import StringType
spark = SparkSession.builder.appName('test').getOrCreate()
from random import randint

In [7]:
df = spark.read.option('header','True').csv('csv/students - Sheet1.csv')
df.show()

+------+----------+---------+-------------+--------------------+------+-------------------+
|    id|first_name|last_name|date_of_birth|           ethnicity|gender|              email|
+------+----------+---------+-------------+--------------------+------+-------------------+
|111111|      John|      Doe|   01-01-2000|            Hispanic|     M|   jdoe@example.com|
|111112|      Jane|    Smith|   01-05-2001|            Hispanic|     F| jsmith@example.com|
|111113|     Sarah|   Thomas|   02-01-2002|            Hispanic|     M|sthomas@example.com|
|111114|     Frank|    Brown|   01-03-2002|Race/ethnicity un...|     M| fbrown@example.com|
|111115|      Mike|    Davis|   03-01-2001|               White|     F| mdavis@example.com|
|111116|  Jennifer|   Wilson|   01-01-2002|               Asian|     M|jwilson@example.com|
|111117|   Jessica|   Garcia|   01-01-2000|               White|     F|jgarcia@example.com|
|111118|      Fred|    Clark|   01-07-1999|            Hispanic|     F| fclark@e

In [8]:
df = df.withColumn("FullName",concat_ws(" ",df.first_name,df.last_name))
df.show()

+------+----------+---------+-------------+--------------------+------+-------------------+---------------+
|    id|first_name|last_name|date_of_birth|           ethnicity|gender|              email|       FullName|
+------+----------+---------+-------------+--------------------+------+-------------------+---------------+
|111111|      John|      Doe|   01-01-2000|            Hispanic|     M|   jdoe@example.com|       John Doe|
|111112|      Jane|    Smith|   01-05-2001|            Hispanic|     F| jsmith@example.com|     Jane Smith|
|111113|     Sarah|   Thomas|   02-01-2002|            Hispanic|     M|sthomas@example.com|   Sarah Thomas|
|111114|     Frank|    Brown|   01-03-2002|Race/ethnicity un...|     M| fbrown@example.com|    Frank Brown|
|111115|      Mike|    Davis|   03-01-2001|               White|     F| mdavis@example.com|     Mike Davis|
|111116|  Jennifer|   Wilson|   01-01-2002|               Asian|     M|jwilson@example.com|Jennifer Wilson|
|111117|   Jessica|   Garcia

In [9]:
listofcolumns = df.columns
print(listofcolumns)
for colmn in listofcolumns:
    df = df.withColumnRenamed(colmn,colmn.capitalize())
df = df.select('Id','First_name','Last_name','Fullname','Gender','Ethnicity','Date_of_birth','Email').withColumn('Salary',lit(None))
df = df.withColumn('Salary',when(df.Gender=='M',randint(25000,35000)).when(df.Gender=='F',randint(15000,24000)))
df.show()
# df.select(to_date(col('Date_of_birth'),"dd-MM-yyyy")).show()

['id', 'first_name', 'last_name', 'date_of_birth', 'ethnicity', 'gender', 'email', 'FullName']


+------+----------+---------+---------------+------+--------------------+-------------+-------------------+------+
|    Id|First_name|Last_name|       Fullname|Gender|           Ethnicity|Date_of_birth|              Email|Salary|
+------+----------+---------+---------------+------+--------------------+-------------+-------------------+------+
|111111|      John|      Doe|       John Doe|     M|            Hispanic|   01-01-2000|   jdoe@example.com| 30270|
|111112|      Jane|    Smith|     Jane Smith|     F|            Hispanic|   01-05-2001| jsmith@example.com| 22162|
|111113|     Sarah|   Thomas|   Sarah Thomas|     M|            Hispanic|   02-01-2002|sthomas@example.com| 30270|
|111114|     Frank|    Brown|    Frank Brown|     M|Race/ethnicity un...|   01-03-2002| fbrown@example.com| 30270|
|111115|      Mike|    Davis|     Mike Davis|     F|               White|   03-01-2001| mdavis@example.com| 22162|
|111116|  Jennifer|   Wilson|Jennifer Wilson|     M|               Asian|   01-0

In [10]:
def give_ranking(salary,gender):
    if gender == 'M' and salary > 24000:
        return "level1"
    elif gender == 'M' and salary <=24000:
        return 'level2'
    elif gender == 'F' and salary > 15000:
        return 'level1'
    elif gender == 'F' and salary <=15000:
        return 'level2'
    else:
        return None
df.printSchema()
update_level = udf(lambda s,g:give_ranking(s,g),StringType())
df.show()
# df_1 = df.withColumn('Ranking',update_level(df['Salary'],df['Gender']))
# df_1.show()

root
 |-- Id: string (nullable = true)
 |-- First_name: string (nullable = true)
 |-- Last_name: string (nullable = true)
 |-- Fullname: string (nullable = false)
 |-- Gender: string (nullable = true)
 |-- Ethnicity: string (nullable = true)
 |-- Date_of_birth: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Salary: integer (nullable = true)

+------+----------+---------+---------------+------+--------------------+-------------+-------------------+------+
|    Id|First_name|Last_name|       Fullname|Gender|           Ethnicity|Date_of_birth|              Email|Salary|
+------+----------+---------+---------------+------+--------------------+-------------+-------------------+------+
|111111|      John|      Doe|       John Doe|     M|            Hispanic|   01-01-2000|   jdoe@example.com| 30270|
|111112|      Jane|    Smith|     Jane Smith|     F|            Hispanic|   01-05-2001| jsmith@example.com| 22162|
|111113|     Sarah|   Thomas|   Sarah Thomas|     M|         

In [16]:
import datetime
a = "2000-01-01"
a = datetime.datetime.strptime(a, "%Y-%d-%m").date()
print(type(datetime.date.today()-a))



<class 'datetime.timedelta'>
